In [1]:
import config
import os

In [2]:
from narrative_llm_agent.agents.kbase_agent import KBaseAgent
from narrative_llm_agent.agents.analyst import AnalystAgent
from narrative_llm_agent.agents.KnowledgeGraph import KGAgent
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process


In [3]:
api_key = os.environ["OPENAI_API_KEY"]
neo4j_uri = os.environ["NEO4J_URI"] 
neo4j_username  = os.environ["NEO4J_USERNAME"]
neo4j_password = os.environ["NEO4J_PASSWORD"]

In [4]:
token = 'dumy_token'

In [5]:
def test_kbase_agent():
    agent = KBaseAgent("foo", None)
    assert agent._token == "foo"
    print("like totally passed.")

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-4",streaming=True)

In [7]:
researcher = AnalystAgent(token=token, llm=llm)
KGer = KGAgent(token=token, llm=llm)

In [8]:
# # Create tasks for your agents
# task1 = Task(
#   description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.  
#   Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them? 
#   Use the Kbase App catalog and documentation to come up with a list of apps to use for each step.
#   Use the knowledge graph to make sure that the apps that you suggest exist
#   Get the approval of a human.
# """,
#   agent=researcher.agent
# )
# task2 = Task(
#   description="""I am trying to use the trimmomatic and gtdb apps, are there any common data objects between the apps?
# """,
#   agent=researcher.agent
# )

In [12]:
# task1 = Task(
#   description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing.  
#   Using KBase apps, I would like to understand if my newly sequenced microbe is a cysteine auxotroph. Which KBase apps should I use and how do I use them? 
#   """,
#   agent=researcher.agent
# )
task1 = Task (description="""I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing tand would like to check its quality.
    What app should I use. Please suggest only one app at a time for a particular step. If you must suggest alternative apps please limit to only one.?""", agent=researcher.agent)
task2 = Task(
  description="""
  Use the knowledge graph to make sure that the apps that you suggest exist
  After you check its availability in the Knowledge Graph, ask a human if they approve of the suggested app. If not check for the additional recommended app in the Knowledge Graph.""",
  agent=KGer.agent
)

In [13]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher.agent,KGer.agent],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

Overriding of current TracerProvider is not allowed


In [14]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: Computational Biologist and Geneticist
[INFO]: Starting Task: I have sequenced a new microbe. I have a fastq file of reads from paired end illumina sequencing tand would like to check its quality.
    What app should I use. Please suggest only one app at a time for a particular step. If you must suggest alternative apps please limit to only one.?


> Entering new CrewAgentExecutor chain...
Use Tool: KBase app catalog retrieval tool
Input: What is the best app in KBase for checking the quality of a fastq file from paired end illumina sequencing? 

{'query': 'What is the best app in KBase for checking the quality of a fastq file from paired end illumina sequencing?', 'chat_history': '', 'result': 'The "Reads Utils - v1.0.2" app in KBase could be used to check the quality of a fastq file from paired end Illumina sequencing. This app can convert libraries\' quality scores from phred64 to phred33, which can be useful for quality checking. However, for a more comprehe

 None. Look for quality control apps like fast qc


 

None. Look for quality control apps like fast qc

Use Tool: KG retrieval tool
Input: "FastQC"

> Entering new AgentExecutor chain...

Invoking: `Information` with `{'entity': 'FastQC', 'entity_type': 'AppCatalog'}`


Need additional information, which of these did you mean: 
{'candidate': 'Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN'}
{'candidate': 'Assess Read Quality with FastQC - v0.12.1'}
{'candidate': 'SINGLE END READS'}I found multiple KBase apps related to FastQC. Could you please specify which one you are interested in? Here are the options:

1. Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN
2. Assess Read Quality with FastQC - v0.12.1
3. SINGLE END READS

> Finished chain.
 

I found multiple KBase apps related to FastQC. Could you please specify which one you are interested in? Here are the options:

1. Join Overlapping Mate Pairs with ea-utils FASTQ-JOIN
2. Assess Read Quality with FastQC - v0.12.1
3. SINGLE END READS

Use Tool: human
Input: I found multipl

 2


 

2

Final Answer: The "Assess Read Quality with FastQC - v0.12.1" app is available in KBase and has been approved for use.

> Finished chain.
[DEBUG]: [Knowledge Graph retrieval] Task output: The "Assess Read Quality with FastQC - v0.12.1" app is available in KBase and has been approved for use.


######################
The "Assess Read Quality with FastQC - v0.12.1" app is available in KBase and has been approved for use.
